In [31]:
# Step 1: Install and Import Required Libraries
!pip install aif360 scikit-learn matplotlib pandas

# General libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Scikit-learn libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# AIF360 libraries
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover, LFR

# Step 2: Load and Preprocess the Dataset
df = pd.read_csv('stackoverflow_full.csv')

# Encode categorical features
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])  # Man=1, Woman=0

# Convert 'Age' to numeric and simplify to binary categories
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Age'] = df['Age'].apply(lambda x: 0 if x < 35 else 1)
df['Age'].fillna(df['Age'].median(), inplace=True)

# Encode 'EdLevel' (Education Level)
le_education = LabelEncoder()
df['EdLevel'] = le_education.fit_transform(df['EdLevel'])  # Convert education levels to numeric

# Select features and target
features = ['Gender', 'Age', 'EdLevel', 'YearsCodePro']
target = 'Employed'

# Split dataset into features (X) and target (y)
X = df[features]
y = df[target]

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Re-attach the target column to the train and test DataFrames
train_df = pd.DataFrame(X_train, columns=features)
train_df[target] = y_train.values

test_df = pd.DataFrame(X_test, columns=features)
test_df[target] = y_test.values

# Convert data into AIF360's BinaryLabelDataset format
train_data = BinaryLabelDataset(df=train_df, label_names=[target],
                                protected_attribute_names=['Gender', 'Age'],
                                favorable_label=1, unfavorable_label=0)

test_data = BinaryLabelDataset(df=test_df, label_names=[target],
                               protected_attribute_names=['Gender', 'Age'],
                               favorable_label=1, unfavorable_label=0)

In [32]:
# Load the metric class
from aif360.metrics import BinaryLabelDatasetMetric

# Create the metric object
metric_orig_train = BinaryLabelDatasetMetric(
    train_data, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

# Load and create explainers
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
text_exp_orig_train = MetricTextExplainer(metric_orig_train)

# Print statistical parity difference
print("Statistical Parity Difference:")
print(text_exp_orig_train.statistical_parity_difference())

# Calculate and print Direct Impact (Disparate Impact)
direct_impact_orig_train = metric_orig_train.disparate_impact()

print("Direct Impact (Disparate Impact) Before Reweighing (Training Data):")
print(direct_impact_orig_train)

Statistical Parity Difference:
Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): 0.09742853007198843
Direct Impact (Disparate Impact) Before Reweighing (Training Data):
1.2196825990565507


In [33]:
# Step 3: Apply Pre-Processing Techniques for Bias Mitigation

# Option 1: Reweighing Technique
reweighing = Reweighing(unprivileged_groups=[{'Gender': 0}], privileged_groups=[{'Gender': 1}])
train_data_reweighed = reweighing.fit_transform(train_data)

In [34]:
# Import the reweighing preprocessing algorithm class
from aif360.algorithms.preprocessing import Reweighing

# Create the algorithm object
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

# Train and transform the training data
RW.fit(train_data)
train_transf_dataset = RW.transform(train_data)

In [35]:
# Checking for Bias in the Original Data Using SPD

# Create the metric object for pre-processed data
metric_transf_train = BinaryLabelDatasetMetric(
    train_transf_dataset, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

# Create explainer
text_exp_transf_train = MetricTextExplainer(metric_transf_train)

# Print statistical parity difference
print("Statistical Parity Difference After Reweighing (Training Data):")
print(text_exp_transf_train.statistical_parity_difference())


# Calculate and print Direct Impact (Disparate Impact)
direct_impact_transf_train = metric_transf_train.disparate_impact()

print("Direct Impact (Disparate Impact) After Reweighing (Training Data):")
print(direct_impact_transf_train)


Statistical Parity Difference After Reweighing (Training Data):
Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -1.1102230246251565e-16
Direct Impact (Disparate Impact) After Reweighing (Training Data):
0.9999999999999998


In [36]:
# Apply the learned re-weighing pre-processor to the test data
test_transf_dataset = RW.transform(test_data)

# Create metric objects for original and pre-processed test data
metric_orig_test = BinaryLabelDatasetMetric(
    test_data, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
metric_transf_test = BinaryLabelDatasetMetric(
    test_transf_dataset, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

# Create explainers for both metric objects
text_exp_orig_test = MetricTextExplainer(metric_orig_test)
text_exp_transf_test = MetricTextExplainer(metric_transf_test)

# Print statistical parity difference for original and transformed test data
print("Statistical Parity Difference Before Reweighing (Test Data):")
print(text_exp_orig_test.statistical_parity_difference())

print("Statistical Parity Difference After Reweighing (Test Data):")
print(text_exp_transf_test.statistical_parity_difference())

# Calculate and print Direct Impact (Disparate Impact) for original and transformed test data
direct_impact_orig_test = metric_orig_test.disparate_impact()
direct_impact_transf_test = metric_transf_test.disparate_impact()

print("Direct Impact (Disparate Impact) Before Reweighing (Test Data):")
print(direct_impact_orig_test)

print("Direct Impact (Disparate Impact) After Reweighing (Test Data):")
print(direct_impact_transf_test)


Statistical Parity Difference Before Reweighing (Test Data):
Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): 0.06996222752788511
Statistical Parity Difference After Reweighing (Test Data):
Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.02740185124008554
Direct Impact (Disparate Impact) Before Reweighing (Test Data):
1.1488018374006983
Direct Impact (Disparate Impact) After Reweighing (Test Data):
0.951331485897656
